In [1]:
import numpy as np
import pandas as pd 
import keras
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, Lambda, GlobalAveragePooling2D
from keras.preprocessing import image

import os
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
from keras.applications import xception
from keras.applications import inception_v3, resnet50

In [3]:
data_dir = os.getcwd()
df_train = pd.read_csv('labels.csv')
df_test = pd.read_csv('sample_submission.csv')

In [4]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


In [5]:
target_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(target_series, sparse=True)

In [6]:
one_hot_labels = np.asarray(one_hot)

In [7]:
def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string
        train_or_test: string 'train' or 'test'.
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    img = image.load_img(os.path.join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    #img = image.img_to_array(img)
    return img

In [8]:
IM_SIZE = 299

In [9]:
x_train = np.zeros((len(df_train), IM_SIZE, IM_SIZE, 3), dtype=np.uint8)
y_train = np.zeros((one_hot_labels.shape), dtype=np.uint8)
for i, img_id in tqdm(enumerate(df_train['id'])):
    img = read_img(img_id, 'train', (IM_SIZE, IM_SIZE))
    x_train[i] = img
    y_train[i] = one_hot_labels[i]
    
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

10222it [00:54, 187.26it/s]

Train Images shape: (10222, 299, 299, 3) size: 2,741,571,066


In [10]:
print(y_train.shape, x_train.shape)

(10222, 120) (10222, 299, 299, 3)


In [11]:
num_class = y_train.shape[1]

### Extract Xception and Inception bottleneck features

In [12]:
def get_features(MODEL, data=x_train):
    cnn_model = MODEL(include_top=False, input_shape=(IM_SIZE, IM_SIZE, 3), weights='imagenet')
    
    inputs = Input((IM_SIZE, IM_SIZE, 3))
    x = inputs
    x = Lambda(preprocess_input, name='preprocessing')(x)
    x = cnn_model(x)
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs, x)

    features = cnn_model.predict(data, batch_size=64, verbose=1)
    return features

In [13]:
inception_features = get_features(inception_v3.InceptionV3, x_train)
xception_features = get_features(xception.Xception, x_train)


10222/10222 [==============================] - 4087s   


In [32]:
resnet_features = get_features(resnet50.ResNet50, x_train)

10222/10222 [==============================] - 2057s  


In [33]:
features = np.concatenate([inception_features, xception_features, resnet_features], axis=-1)

### Model training

In [55]:
sgd = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-2)
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                          patience=10, verbose=1),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, verbose=1)]

In [56]:
inputs = Input(features.shape[1:])
x = inputs
x = Dropout(0.5)(x)
x = Dense(num_class, activation='softmax')(x)
model = Model(inputs, x)

In [57]:
model.compile(loss='categorical_crossentropy', optimizer=sgd,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 6144)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 6144)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 120)               737400    
Total params: 737,400
Trainable params: 737,400
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.fit(features, y_train, batch_size=128, epochs=150, validation_split=0.1, verbose=1, callbacks=callbacks)

Train on 9199 samples, validate on 1023 samples
Epoch 1/150
9199/9199 [==============================] - 2s - loss: 3.3606 - acc: 0.3608 - val_loss: 1.5680 - val_acc: 0.8231
Epoch 2/150
9199/9199 [==============================] - 0s - loss: 1.1878 - acc: 0.8219 - val_loss: 0.8055 - val_acc: 0.8866
Epoch 3/150
9199/9199 [==============================] - 0s - loss: 0.7643 - acc: 0.8702 - val_loss: 0.6154 - val_acc: 0.8964
Epoch 4/150
9199/9199 [==============================] - 0s - loss: 0.6174 - acc: 0.8907 - val_loss: 0.5318 - val_acc: 0.9120
Epoch 5/150
9199/9199 [==============================] - 0s - loss: 0.5481 - acc: 0.8980 - val_loss: 0.4869 - val_acc: 0.9159
Epoch 6/150
9199/9199 [==============================] - 0s - loss: 0.5043 - acc: 0.9018 - val_loss: 0.4625 - val_acc: 0.9101
Epoch 7/150
9199/9199 [==============================] - 0s - loss: 0.4788 - acc: 0.9040 - val_loss: 0.4372 - val_acc: 0.9120
Epoch 8/150
9199/9199 [==============================] - 0s - loss: 0.

### Testing

In [44]:
x_test = np.zeros((len(df_test), IM_SIZE, IM_SIZE, 3), dtype='float32')
for i, img_id in tqdm(enumerate(df_test['id'])):
    img = read_img(img_id, 'test', (IM_SIZE, IM_SIZE))
    x_test[i] = img
    
print('Test Images shape: {} size: {:,}'.format(x_test.shape, x_test.size))

10357it [00:28, 364.76it/s]

Test Images shape: (10357, 299, 299, 3) size: 2,777,778,471


In [59]:
test_x_features = get_features(xception.Xception, x_test)
test_i_features = get_features(inception_v3.InceptionV3, x_test)
test_resnet_features = get_features(resnet50.ResNet50, x_test)
test_features = np.concatenate([test_i_features, test_x_features, test_resnet_features], axis=-1)

10357/10357 [==============================] - 1971s   


In [60]:
y_pred = model.predict(test_features, batch_size=128)

In [61]:
sub = pd.DataFrame(y_pred)
col_names = one_hot.columns.values
sub.columns = col_names

sub.insert(0, 'id', df_test['id'])
sub.head(10)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.000051,0.000033,0.000024,0.000010,0.000018,0.000019,0.000019,0.000009,0.000009,...,0.000023,0.000032,0.000021,0.000013,0.000017,0.000013,0.000017,0.000009,0.000053,0.000034
1,00102ee9d8eb90812350685311fe5890,0.000027,0.000024,0.000041,0.000027,0.000051,0.000168,0.000033,0.000035,0.000036,...,0.000033,0.000031,0.000030,0.000059,0.000054,0.000053,0.000430,0.000026,0.000069,0.000021
2,0012a730dfa437f5f3613fb75efcd4ce,0.000034,0.011641,0.000245,0.000221,0.000103,0.000139,0.000062,0.000075,0.000235,...,0.000118,0.000185,0.000340,0.000509,0.000321,0.001896,0.000049,0.000225,0.000253,0.000153
3,001510bc8570bbeee98c8d80c8a95ec1,0.001762,0.000170,0.000069,0.000024,0.000155,0.000150,0.000017,0.000137,0.000114,...,0.000120,0.000048,0.000052,0.000069,0.000041,0.000021,0.000029,0.000195,0.000008,0.000073
4,001a5f3114548acdefa3d4da05474c2e,0.049060,0.000555,0.000343,0.000039,0.000276,0.000225,0.000328,0.000218,0.000192,...,0.000461,0.000537,0.000462,0.000298,0.000199,0.000100,0.000268,0.000449,0.000432,0.001543
5,00225dcd3e4d2410dd53239f95c0352f,0.000653,0.013143,0.000793,0.004140,0.000384,0.000649,0.000586,0.000864,0.000297,...,0.054726,0.000659,0.000468,0.000428,0.000272,0.000489,0.000256,0.000250,0.004086,0.000621
6,002c2a3117c2193b4d26400ce431eebd,0.000035,0.000053,0.000109,0.000056,0.000048,0.000024,0.986311,0.000067,0.000012,...,0.000027,0.000029,0.000107,0.000033,0.000020,0.000032,0.000043,0.000012,0.000134,0.001218
7,002c58d413a521ae8d1a5daeb35fc803,0.000054,0.000050,0.000079,0.000037,0.000052,0.000120,0.000091,0.000031,0.000044,...,0.000108,0.000034,0.000039,0.000069,0.000039,0.000083,0.001569,0.000036,0.000120,0.000054
8,002f80396f1e3db687c5932d7978b196,0.000007,0.000037,0.000017,0.000031,0.000011,0.000014,0.000004,0.000008,0.000003,...,0.000004,0.000003,0.000021,0.000010,0.000022,0.000006,0.000006,0.000042,0.000046,0.000004
9,0036c6bcec6031be9e62a257b1c3c442,0.000194,0.000369,0.000362,0.000066,0.000194,0.000082,0.000124,0.000041,0.000688,...,0.000178,0.000095,0.000367,0.000081,0.000172,0.000632,0.000196,0.000051,0.000171,0.000107


In [62]:
sub.to_csv('inc_exc_submission1.csv', index=False)

In [63]:
sub.tail()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
10352,ffeda8623d4eee33c6d1156a2ecbfcf8,0.000158,0.002164,0.000474,0.000282,0.000255,0.000304,0.000174,0.000192,0.000610,...,0.007343,0.000584,0.000219,0.000630,0.000501,0.000504,0.000113,0.000489,0.000698,0.000153
10353,fff1ec9e6e413275984966f745a313b0,0.000021,0.000033,0.000072,0.000046,0.000280,0.000020,0.000038,0.000027,0.000150,...,0.000021,0.000030,0.006166,0.000074,0.972087,0.000023,0.000014,0.000121,0.000059,0.000065
10354,fff74b59b758bbbf13a5793182a9bbe4,0.000077,0.000123,0.001822,0.000045,0.000030,0.000090,0.000164,0.000141,0.000054,...,0.000171,0.000101,0.000122,0.000132,0.000043,0.000125,0.000071,0.000067,0.000157,0.000070
10355,fff7d50d848e8014ac1e9172dc6762a3,0.000298,0.000099,0.000036,0.000017,0.000078,0.000041,0.000080,0.000034,0.000037,...,0.000866,0.000047,0.000082,0.000058,0.000018,0.000015,0.000130,0.000030,0.000138,0.000225
10356,fffbff22c1f51e3dc80c4bf04089545b,0.000119,0.001478,0.000398,0.000130,0.000224,0.000302,0.000085,0.000084,0.000083,...,0.000059,0.000076,0.000145,0.000233,0.000214,0.000065,0.000100,0.001715,0.000367,0.000059
